In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [6]:
import sys
sys.path.append('../')

from myutils.metrics import mapk

In [7]:
OUTPUT_NAME = 'image_embedding'

In [8]:
train_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_log.csv')
train_df = pd.read_csv('../datasets/atmaCup16_Dataset/train_label.csv')
test_log_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_log.csv')
yado_df = pd.read_csv('../datasets/atmaCup16_Dataset/yado.csv')

train_session_only_df = train_df[['session_id']]
test_session_only_df = pd.read_csv('../datasets/atmaCup16_Dataset/test_session.csv')

train_test_log_df = pd.concat([train_log_df, test_log_df], axis=0).reset_index(drop=True)

In [9]:
image_parquet = pd.read_parquet('../datasets/atmaCup16_Dataset/image_embeddings.parquet')

In [10]:
image_parquet

,yad_no,category,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,1,exterior,-0.010736,-0.055452,-0.034968,0.052371,0.013423,0.037117,0.098775,0.074732,...,-0.039645,0.047950,-0.001786,-0.054268,-0.173725,-0.037862,-0.039419,0.039463,-0.071842,-0.007374
1,1,exterior,0.056188,-0.026121,-0.056543,0.022742,-0.046260,0.020954,0.006757,-0.010403,...,-0.001135,0.019452,-0.158739,0.069106,-0.314221,-0.014093,0.020678,0.058678,0.027612,-0.007790
2,1,exterior,0.059149,0.007443,-0.050527,0.031444,-0.034918,0.018043,0.016075,-0.000287,...,-0.017606,0.028306,-0.153259,0.079555,-0.305520,-0.034372,0.037976,0.065278,0.023551,-0.006770
3,1,facility,-0.008016,0.031106,-0.013940,0.013397,-0.025822,0.020344,0.049135,-0.031101,...,-0.029366,-0.035536,-0.098193,0.031702,-0.359215,-0.041691,0.002394,-0.022892,-0.039722,-0.013380
4,1,facility,0.010669,-0.000045,0.002484,0.008006,-0.041051,0.022130,0.026998,0.011513,...,-0.008329,-0.025674,-0.175958,0.052703,-0.350555,-0.035050,-0.029618,0.010537,-0.040529,-0.031906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178401,13806,others,-0.034216,-0.006210,0.010540,0.019159,-0.096793,0.036234,-0.009786,-0.036364,...,-0.026683,-0.026870,-0.032283,-0.070201,-0.180044,-0.000277,0.045372,0.011902,0.038005,0.035427
178402,13806,others,0.039488,-0.022872,-0.012293,-0.006093,-0.042878,0.002610,-0.002810,0.031854,...,0.011119,0.008077,-0.069928,-0.009791,-0.104985,-0.045665,-0.044150,-0.053934,0.010256,-0.005381
178403,13806,room,-0.029599,-0.031814,-0.020295,0.012060,0.031184,-0.025655,0.000405,0.055843,...,0.038399,0.004924,0.070224,0.006177,-0.344118,-0.028549,-0.015591,-0.032350,-0.020876,0.021666
178404,13806,room,-0.017197,-0.027276,-0.023111,0.019118,-0.032534,-0.016227,0.005969,0.024882,...,0.007044,0.013483,-0.121946,0.034773,-0.332512,-0.032718,-0.027062,-0.015025,0.007962,0.020657


In [7]:
image_parquet = image_parquet.groupby(['yad_no', 'category']).mean().reset_index()

In [8]:
image_parquet

,yad_no,category,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,...,emb_502,emb_503,emb_504,emb_505,emb_506,emb_507,emb_508,emb_509,emb_510,emb_511
0,1,exterior,0.034867,-0.024710,-0.047346,0.035519,-0.022585,0.025371,0.040536,0.021347,...,-0.019462,0.031903,-0.104595,0.031464,-0.264489,-0.028776,0.006412,0.054473,-0.006893,-0.007311
1,1,facility,0.005908,0.022932,-0.002892,0.016683,-0.021860,0.017787,0.049936,-0.006937,...,-0.013551,-0.028419,-0.138819,0.032918,-0.352742,-0.037674,-0.022275,-0.011646,-0.032959,-0.029457
2,1,food,-0.021771,-0.008306,0.001915,0.058299,-0.077523,-0.035897,0.005612,0.033138,...,-0.017894,0.013657,-0.035324,-0.030708,-0.301827,-0.010079,-0.019531,-0.025723,0.014168,-0.025918
3,1,others,0.026194,0.002307,-0.010998,-0.011777,-0.052911,0.000792,-0.003802,0.009565,...,0.007628,-0.003932,-0.009213,-0.030807,-0.274149,-0.038935,0.001598,0.025866,0.036270,0.003328
4,1,room,-0.005005,-0.019154,-0.022597,0.047721,-0.063572,-0.013437,-0.010168,-0.014173,...,-0.007693,0.006168,-0.135753,-0.001837,-0.296622,-0.035557,0.011043,0.014466,0.008492,0.031763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65285,13806,exterior,0.044333,0.002380,-0.033033,0.007966,-0.023937,0.019838,0.006921,-0.001176,...,0.024568,-0.023183,-0.093438,0.025394,-0.314576,-0.017718,0.010373,0.024407,0.011441,0.014382
65286,13806,facility,0.032105,-0.017752,0.007961,-0.008931,-0.033860,0.010914,-0.017122,0.028818,...,0.022236,-0.014914,-0.077984,-0.033132,-0.295392,-0.026531,-0.009152,0.048092,0.016831,0.000526
65287,13806,food,-0.004942,0.026008,0.006781,0.024776,-0.062511,-0.007334,0.009320,0.019746,...,-0.002297,0.003170,-0.087492,0.025231,-0.314974,-0.051805,-0.021892,0.002601,0.032104,-0.037519
65288,13806,others,-0.012096,-0.010845,-0.014088,0.013171,-0.042735,0.019514,-0.005109,0.012759,...,0.008009,-0.021292,-0.069413,-0.029234,-0.201572,-0.034007,-0.001261,0.001400,0.021064,0.015965


In [9]:
dimensions = 512

In [10]:
for category, image_df in image_parquet.groupby('category'):

    yad_to_image_dict = dict(zip(image_df['yad_no'], image_df[[f'emb_{i}' for i in range(dimensions)]].values))
    is_seen_set = set(yad_to_image_dict.keys())

    item_ids = yado_df['yad_no'].to_list()
    # item_factors = {item: model.wv[item] for item in item_ids}
    item_factors = {}
    for item in item_ids:
        if item in is_seen_set:
            item_factors[item] = yad_to_image_dict[item]
        else:
            item_factors[item] = np.zeros(dimensions)
    item_factors_df = pd.DataFrame(item_factors).T.reset_index().rename(columns={"index": "yad_no"})

    item_cols = [f"item_factor_{i}" for i in range(dimensions)]
    item_factors_df.columns = ["yad_no"] + item_cols

    user_item_list_dict = {session_id: yad_nos.tolist() for session_id, yad_nos in train_test_log_df.groupby('session_id')['yad_no']}
    
    # all item
    user_factors = {}
    for user_id, user_item_list in user_item_list_dict.items():
        vecs = []
        for item_id in user_item_list:
            if item_id in is_seen_set:
                vecs.append(yad_to_image_dict[item_id])
        if len(vecs) == 0:
            vecs = np.zeros(dimensions)
        else:
            vecs = np.mean(vecs, axis=0)
        user_factors[user_id] = vecs
    user_factors_df = pd.DataFrame(user_factors).T.reset_index().rename(columns={"index": "session_id"})
    user_cols = [f"user_factor_{i}" for i in range(dimensions)]
    user_factors_df.columns = ["session_id"] + user_cols

    # last item
    user_factors_last_item = {}
    for user_id, user_item_list in user_item_list_dict.items():
        vecs = []
        item_id = user_item_list[-1]
        if item_id in is_seen_set:
            vecs = yad_to_image_dict[item_id]
        else:
            vecs = np.zeros(dimensions)
        user_factors_last_item[user_id] = vecs
    user_factors_last_item_df = pd.DataFrame(user_factors_last_item).T.reset_index().rename(columns={"index": "session_id"})
    user_cols = [f"user_factor_{i}" for i in range(dimensions)]
    user_factors_last_item_df.columns = ["session_id"] + user_cols

    import os
    os.makedirs(f'../datasets/{OUTPUT_NAME}_pkl', exist_ok=True)
    user_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_{category}_user_factors_df.pkl')
    user_factors_last_item_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_{category}_user_factors_last_item_df.pkl')
    item_factors_df.to_pickle(f'../datasets/{OUTPUT_NAME}_pkl/{OUTPUT_NAME}_{category}_item_factors_df.pkl')

In [11]:
image_parquet['category'].unique()

array(['exterior', 'facility', 'food', 'others', 'room'], dtype=object)